# Files that had no segmentation done

In [1]:
missing_mappings_filename = 'missing_mappings_after_restoration.pickle'
nonexistent_meta_filename = 'nonexistent_meta_files_after_restoration.pickle'

In [16]:
import pickle
import shutil
from pathlib import Path

## Missing mappings

In [3]:
with open(missing_mappings_filename, 'rb') as file:
    missing_mappings = pickle.load(file)

In [5]:
filenames = [tup[0] for tup in missing_mappings]

In [15]:
target_dir = Path('/chuvdatarc/ja8818/RAD/LINE/Benedetta_Franceschiello/5_OCTs_Jakub_Gwizdala/0_backup_folder_development/AMD/scans_for_final_segmentation/to_segment/missing_matching_meta')
data_dir = Path('/chuvdatarc/ja8818/RAD/LINE/Benedetta_Franceschiello/5_OCTs_Jakub_Gwizdala/0_backup_folder_development/AMD/raw_dataset')

In [17]:
missing_matching_meta_mapping = {}
for i, filename in enumerate(filenames):
    source_location = data_dir / filename
    group = filename[:filename.find('/')]
    dest_location = target_dir / group / f'{i}_mm.tif'
    missing_matching_meta_mapping[str(source_location)] = str(dest_location)
    shutil.copy2(source_location, dest_location)

In [23]:
with open('mappings/missing_matching_meta_mapping.pickle', 'wb') as file:
    pickle.dump(missing_matching_meta_mapping, file)

## Nonexistent meta

In [24]:
target_dir = Path('/chuvdatarc/ja8818/RAD/LINE/Benedetta_Franceschiello/5_OCTs_Jakub_Gwizdala/0_backup_folder_development/AMD/scans_for_final_segmentation/to_segment/not_done_matching_meta')
data_dir = Path('/chuvdatarc/ja8818/RAD/LINE/Benedetta_Franceschiello/5_OCTs_Jakub_Gwizdala/0_backup_folder_development/AMD/raw_dataset')

In [25]:
with open(nonexistent_meta_filename, 'rb') as file:
    nonexistent_meta = pickle.load(file)

In [26]:
with open('lost_meta_files_candidates.pickle', 'rb') as file:
    lost_meta_candidates = pickle.load(file)

In [35]:
nonexistent_meta_mapping = {}
for i, (scan_filename, meta_filenames) in enumerate(nonexistent_meta):
    source_location = data_dir / scan_filename
    group = scan_filename[:scan_filename.find('/')]
    target_location = target_dir / group / f'{i}_nm.tif'
    
    if not source_location.exists():
        break
    
    nonexistent_meta_mapping[str(source_location)] = str(target_location)
    shutil.copy2(source_location, target_location)
    
    if meta_filenames[0] in lost_meta_candidates:
        source_meta_location = lost_meta_candidates[meta_filenames[0]][0]
        target_meta_location = target_dir / group / f'{i}_nm.meta'
        shutil.copy2(source_meta_location, target_meta_location)

In [42]:
with open('mappings/nonexistent_meta_mapping.pickle', 'wb') as file:
    pickle.dump(nonexistent_meta_mapping, file)

## To review

In [52]:
from preprocessing import *

In [53]:
to_review = ['control_8_right', 'control_8_left', 'control_9_left', 'control_9_right', 'control_10_right', 
             'control_10_left', 'control_11_right', 'control_11_left', 'control_12_right', 'control_12_left',
             'amd_2_left', 'amd_6_left', 'control_0_left']

In [54]:
mapping = '/chuvdatarc/ja8818/RAD/LINE/Benedetta_Franceschiello/5_OCTs_Jakub_Gwizdala/0_backup_folder_development/AMD/filename_to_meta_mapping_extended_and_restored.pickle'

In [55]:
with open(mapping, 'rb') as file:
    mapping = pickle.load(file)

In [66]:
preprocessing_dir = Path('/chuvdatarc/ja8818/RAD/LINE/Benedetta_Franceschiello/5_OCTs_Jakub_Gwizdala/0_backup_folder_development/AMD/preprocessing/preprocessed_octs')
meta_root_dir = Path('/chuvdatarc/ja8818/RAD/LINE/Benedetta_Franceschiello/5_OCTs_Jakub_Gwizdala/1_OCT_Dataset/1_Segmented_OCTs/1_Final_Segmented_OCTs')
target_dir = Path('/chuvdatarc/ja8818/RAD/LINE/Benedetta_Franceschiello/5_OCTs_Jakub_Gwizdala/0_backup_folder_development/AMD/scans_for_final_segmentation/to_review')

In [69]:
to_review_mapping = {}
i = 0
for sample in to_review:
    pickle_file = preprocessing_dir / f'{sample}.pickle'
    with open(pickle_file, 'rb') as file:
        preprocessed_scans = pickle.load(file)
    
    for scan in preprocessed_scans:
        scan_data = scan.scan_data()
        scan_source_location = scan_data.oct_scan.filename
        group = scan_data.group
        
        lookup_filename = scan_source_location[scan_source_location.find('raw_dataset') + 12:]
        meta_filename = mapping[lookup_filename][-1]
        meta_source_location = meta_root_dir / meta_filename
        
        scan_target_location = target_dir / group / f'{i}_tr.tif'
        shutil.copy2(scan_source_location, scan_target_location)
        shutil.copy2(meta_source_location, target_dir / group / f'{i}_tr.meta')
        to_review_mapping[scan_source_location] = str(scan_target_location)
        i += 1

In [72]:
with open('mappings/to_review_mapping.pickle', 'wb') as file:
    pickle.dump(to_review_mapping, file)